# NN Fingerprint Analysis

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv("qm9_fp_U0.csv")

# Columns:
# 0 = U0 (target)
# 1 = SMILES
# 2+ = Fingerprints

# Target
y = df.iloc[:, 0].values.astype(float)

# Drop SMILES (col 1)
X = df.iloc[:, 2:].values.astype(float)

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (129012, 2048)
y shape: (129012,)


In [3]:
# First: 80% / 20%
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.20, random_state=42, shuffle=True
)

# Split the 20% temp into 10% val + 10% test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,   # half of 20% → 10%
    random_state=42,
    shuffle=True
)

print("Train:", X_train.shape)
print("Val:", X_val.shape)
print("Test:", X_test.shape)


Train: (103209, 2048)
Val: (12901, 2048)
Test: (12902, 2048)


In [ ]:
import os, time, json, numpy as np, pandas as pd, torch, torch.nn as nn
from pathlib import Path
from itertools import product
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

SEED = 42
np.random.seed(SEED); torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---- scale X on train only; scale y too ----
x_scaler = StandardScaler().fit(X_train.astype(np.float32))
Xtr = x_scaler.transform(X_train.astype(np.float32))
Xva = x_scaler.transform(X_val.astype(np.float32))
Xte = x_scaler.transform(X_test.astype(np.float32))

y_scaler = StandardScaler().fit(y_train.reshape(-1,1).astype(np.float32))
ytr = y_scaler.transform(y_train.reshape(-1,1).astype(np.float32)).astype(np.float32)
yva = y_scaler.transform(y_val.reshape(-1,1).astype(np.float32)).astype(np.float32)
yte = y_scaler.transform(y_test.reshape(-1,1).astype(np.float32)).astype(np.float32)

print("y_train mean/std:", y_train.mean(), y_train.std())

# ---- model ----
class FFN(nn.Module):
    def __init__(self, d_in, hidden, act="relu", p=0.0):
        super().__init__()
        A = nn.ReLU if act=="relu" else nn.GELU
        layers=[]; prev=d_in
        for h in hidden:
            layers += [nn.Linear(prev,h), A(), nn.Dropout(p)]
            prev=h
        layers += [nn.Linear(prev,1)]
        self.net = nn.Sequential(*layers)
    def forward(self,x): return self.net(x)

def loaders(Xtr, ytr, Xva, yva, bs):
    ds_tr = TensorDataset(torch.from_numpy(Xtr), torch.from_numpy(ytr))
    ds_va = TensorDataset(torch.from_numpy(Xva), torch.from_numpy(yva))
    dl_tr = DataLoader(ds_tr, batch_size=bs, shuffle=True, pin_memory=True)
    dl_va = DataLoader(ds_va, batch_size=max(bs, 2048), shuffle=False, pin_memory=True)
    return dl_tr, dl_va

def train_cfg(cfg):
    hidden   = cfg["hidden"]
    act      = cfg["act"]
    p        = cfg["dropout"]
    wd       = cfg["weight_decay"]
    lr       = cfg["lr"]
    bs       = cfg["batch_size"]
    max_ep   = cfg["epochs"]
    patience = cfg["patience"]

    dl_tr, dl_va = loaders(Xtr, ytr, Xva, yva, bs)
    model = FFN(Xtr.shape[1], hidden, act, p).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=3)
    crit = nn.MSELoss()
    scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

    best = {"rmse_val": float("inf"), "state": None, "epoch": -1}
    wait = 0; t0 = time.perf_counter()

    for ep in range(1, max_ep+1):
        model.train(); tr_loss=0.0; nb=0
        for xb,yb in dl_tr:
            xb,yb = xb.to(device), yb.to(device)
            opt.zero_grad(set_to_none=True)
            with torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
                pred = model(xb); loss = crit(pred,yb)
            scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
            tr_loss += loss.item(); nb += 1

        # val
        model.eval(); mse=0.0; n=0
        with torch.no_grad(), torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
            for xb,yb in dl_va:
                xb,yb = xb.to(device), yb.to(device)
                pred = model(xb)
                mse += ((pred-yb)**2).mean().item(); n+=1
        mse /= max(1,n); rmse_val = float(np.sqrt(mse))
        sched.step(mse)

        if rmse_val < best["rmse_val"] - 1e-6:
            best.update({"rmse_val": rmse_val, "state": model.state_dict(), "epoch": ep})
            wait = 0
        else:
            wait += 1
            if wait >= patience: break

    secs = time.perf_counter() - t0
    return best["rmse_val"], best["state"], secs, best["epoch"]

# ---- grid ----
grid = {
    "hidden":      [[512,256], [1024,512,256]],
    "act":         ["relu", "gelu"],
    "dropout":     [0.0, 0.1],
    "weight_decay":[1e-5, 1e-4],
    "lr":          [1e-3, 5e-4],
    "batch_size":  [1024, 2048],
    "epochs":      [100],
    "patience":    [10],
}

def configs(g):
    keys = list(g.keys())
    for vals in product(*[g[k] for k in keys]):
        yield dict(zip(keys, vals))

# ---- streaming search (resumable) ----
out = Path("artifacts_nn"); out.mkdir(exist_ok=True)
csv = out / "nn_val_search_stream.csv"
done = set()
if csv.exists():
    prev = pd.read_csv(csv)
    for _,r in prev.iterrows():
        d = {k: r[k] for k in ["hidden","act","dropout","weight_decay","lr","batch_size","epochs","patience"]}
        done.add(json.dumps(d, sort_keys=True))

for cfg in configs(grid):
    key = json.dumps(cfg, sort_keys=True)
    if key in done:
        print("Skip:", cfg); continue
    print("Train:", cfg)
    rmse_val, state, secs, best_ep = train_cfg(cfg)
    row = dict(cfg); row.update({"rmse_val": rmse_val, "secs": secs, "best_epoch": best_ep})
    pd.DataFrame([row]).to_csv(csv, mode="a", header=not csv.exists(), index=False)
    print(f"-> val RMSE {rmse_val:.4f} | {secs/60:.2f} min | best ep {best_ep}")

# ---- pick best, retrain on train+val, test (inverse-transform reporting) ----
all_df = pd.read_csv(csv).sort_values("rmse_val").reset_index(drop=True)
best = all_df.iloc[0].to_dict()
print("\nBest config:", best)

# retrain best on train+val
Xtv = np.vstack([Xtr, Xva]); ytv = np.vstack([ytr, yva])
bs = int(best["batch_size"])
dl_tv = DataLoader(TensorDataset(torch.from_numpy(Xtv), torch.from_numpy(ytv)),
                   batch_size=bs, shuffle=True, pin_memory=True)
dl_te = DataLoader(TensorDataset(torch.from_numpy(Xte), torch.from_numpy(yte)),
                   batch_size=max(bs,2048), shuffle=False, pin_memory=True)

model = FFN(
    Xtr.shape[1],
    hidden=eval(best["hidden"]) if isinstance(best["hidden"], str) else best["hidden"],
    act=best["act"],
    p=float(best["dropout"])
).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=float(best["lr"]), weight_decay=float(best["weight_decay"]))
crit = nn.MSELoss(); scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

E = int(best["best_epoch"]) + 5
for ep in range(E):
    model.train()
    for xb,yb in dl_tv:
        xb,yb = xb.to(device), yb.to(device)
        opt.zero_grad(set_to_none=True)
        with torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
            pred = model(xb); loss = crit(pred,yb)
        scaler.scale(loss).backward(); scaler.step(opt); scaler.update()

# test (inverse-transform to original units)
model.eval(); preds=[]
with torch.no_grad(), torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
    for xb,yb in dl_te:
        xb = xb.to(device)
        preds.append(model(xb).cpu().numpy())
y_pred_scaled = np.vstack(preds).ravel()

y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).ravel()
y_true = y_scaler.inverse_transform(yte).ravel()
rmse_test = np.sqrt(np.mean((y_pred - y_true)**2))
mae_test  = np.mean(np.abs(y_pred - y_true))
ss_res = np.sum((y_pred - y_true)**2)
ss_tot = np.sum((y_true - y_true.mean())**2)
r2_test  = 1 - ss_res/ss_tot

print(f"\n=== NN Test ===\nRMSE {rmse_test:.6f} | MAE {mae_test:.6f} | R^2 {r2_test:.6f}")

import joblib
joblib.dump({"x_scaler": x_scaler, "y_scaler": y_scaler,
             "state_dict": model.state_dict(), "config": best},
            out / "nn_best_model.joblib")
pd.DataFrame({"y_true": y_true, "y_pred": y_pred}).to_csv(out / "test_preds.csv", index=False)


Device: cuda
y_train mean/std: -11178.274191566177 1087.6680575550738
Train: {'hidden': [512, 256], 'act': 'relu', 'dropout': 0.0, 'weight_decay': 1e-05, 'lr': 0.001, 'batch_size': 1024, 'epochs': 100, 'patience': 10}


C:\Users\colli\AppData\Local\Temp\ipykernel_56612\2527933352.py:61: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))


-> val RMSE 0.4040 | 0.65 min | best ep 33
Train: {'hidden': [512, 256], 'act': 'relu', 'dropout': 0.0, 'weight_decay': 1e-05, 'lr': 0.001, 'batch_size': 2048, 'epochs': 100, 'patience': 10}
-> val RMSE 0.4131 | 0.45 min | best ep 22
Train: {'hidden': [512, 256], 'act': 'relu', 'dropout': 0.0, 'weight_decay': 1e-05, 'lr': 0.0005, 'batch_size': 1024, 'epochs': 100, 'patience': 10}
-> val RMSE 0.4121 | 0.58 min | best ep 29
Train: {'hidden': [512, 256], 'act': 'relu', 'dropout': 0.0, 'weight_decay': 1e-05, 'lr': 0.0005, 'batch_size': 2048, 'epochs': 100, 'patience': 10}
-> val RMSE 0.4190 | 0.77 min | best ep 45
Train: {'hidden': [512, 256], 'act': 'relu', 'dropout': 0.0, 'weight_decay': 0.0001, 'lr': 0.001, 'batch_size': 1024, 'epochs': 100, 'patience': 10}
-> val RMSE 0.3999 | 0.60 min | best ep 30
Train: {'hidden': [512, 256], 'act': 'relu', 'dropout': 0.0, 'weight_decay': 0.0001, 'lr': 0.001, 'batch_size': 2048, 'epochs': 100, 'patience': 10}
-> val RMSE 0.4125 | 0.55 min | best ep 2

C:\Users\colli\AppData\Local\Temp\ipykernel_56612\2527933352.py:153: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  crit = nn.MSELoss(); scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))



=== NN Test ===
RMSE 412.080231 | MAE 230.225250 | R^2 0.857525


In [ ]:
import numpy as np, torch, torch.nn as nn, time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.utils.data import TensorDataset, DataLoader

SEED = 42
np.random.seed(SEED); torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ==== 0) Load your splits ====
# If you already have X_train, y_train, X_val, y_val, X_test, y_test in memory, skip this section.
# Otherwise, uncomment and point to your files (CSV/NPY). Assumes features in X_* and targets in y_*.
X_train = np.load("X_train.npy"); y_train = np.load("y_train.npy")
X_val   = np.load("X_val.npy");   y_val   = np.load("y_val.npy")
X_test  = np.load("X_test.npy");  y_test  = np.load("y_test.npy")

assert all(v in globals() for v in ["X_train","y_train","X_val","y_val","X_test","y_test"]), "Define your splits first."

# ==== 1) Scale X on train only; scale y on train only ====
x_scaler = StandardScaler().fit(X_train.astype(np.float32))
Xtr = x_scaler.transform(X_train.astype(np.float32))
Xva = x_scaler.transform(X_val.astype(np.float32))
Xte = x_scaler.transform(X_test.astype(np.float32))

y_scaler = StandardScaler().fit(y_train.reshape(-1,1).astype(np.float32))
ytr = y_scaler.transform(y_train.reshape(-1,1).astype(np.float32)).astype(np.float32)
yva = y_scaler.transform(y_val.reshape(-1,1).astype(np.float32)).astype(np.float32)
yte = y_scaler.transform(y_test.reshape(-1,1).astype(np.float32)).astype(np.float32)

# ==== 2) Dataloaders ====
def make_loader(X, y, bs, shuffle):
    ds = TensorDataset(torch.from_numpy(X), torch.from_numpy(y))
    return DataLoader(ds, batch_size=bs, shuffle=shuffle, pin_memory=True)

BS = 1024
dl_tr = make_loader(Xtr, ytr, BS, True)
dl_va = make_loader(Xva, yva, max(BS, 2048), False)
dl_te = make_loader(Xte, yte, max(BS, 2048), False)

# ==== 3) Model ====
class FFN(nn.Module):
    def __init__(self, d_in, hidden=(1024,512,256), p=0.1, act="gelu"):
        super().__init__()
        A = nn.GELU if act=="gelu" else nn.ReLU
        layers=[]; prev=d_in
        for h in hidden:
            layers += [nn.Linear(prev,h), A(), nn.Dropout(p)]
            prev=h
        layers += [nn.Linear(prev,1)]
        self.net = nn.Sequential(*layers)
    def forward(self,x): return self.net(x)

model = FFN(Xtr.shape[1], hidden=(1024,512,256), p=0.1, act="gelu").to(device)

# ==== 4) Train w/ early stopping ====
LR = 5e-4
WD = 1e-4
EPOCHS = 100
PATIENCE = 10
opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=3)

crit = nn.MSELoss()
scaler_amp = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

best = {"rmse_val": float("inf"), "state": None, "epoch": -1}
wait = 0
t0 = time.perf_counter()

def eval_rmse_scaled(m, loader):
    m.eval()
    mse = 0.0; n = 0
    with torch.no_grad(), torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
        for xb,yb in loader:
            xb,yb = xb.to(device), yb.to(device)
            pred = m(xb)
            mse += ((pred-yb)**2).mean().item(); n += 1
    return float(np.sqrt(mse / max(1,n)))

for ep in range(1, EPOCHS+1):
    model.train(); nb=0; tr_mse=0.0
    for xb,yb in dl_tr:
        xb,yb = xb.to(device), yb.to(device)
        opt.zero_grad(set_to_none=True)
        with torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
            pred = model(xb); loss = crit(pred,yb)
        scaler_amp.scale(loss).backward(); scaler_amp.step(opt); scaler_amp.update()
        tr_mse += loss.item(); nb += 1

    rmse_val_scaled = eval_rmse_scaled(model, dl_va)
    sched.step(rmse_val_scaled**2)

    if rmse_val_scaled < best["rmse_val"] - 1e-6:
        best.update({"rmse_val": rmse_val_scaled, "state": model.state_dict(), "epoch": ep})
        wait = 0
    else:
        wait += 1
        if wait >= PATIENCE: break

elapsed = time.perf_counter() - t0
print(f"Best epoch: {best['epoch']} | val RMSE (scaled): {best['rmse_val']:.4f} | time: {elapsed/60:.2f} min")

# Restore best
model.load_state_dict(best["state"])

# ==== 5) Evaluate on Train / Val / Test in ORIGINAL UNITS ====
def evaluate_original_units(m, X, y_scaled, y_true):
    m.eval(); preds=[]
    loader = DataLoader(TensorDataset(torch.from_numpy(X), torch.from_numpy(y_scaled)),
                        batch_size=max(BS,2048), shuffle=False, pin_memory=True)
    with torch.no_grad(), torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
        for xb,_ in loader:
            xb = xb.to(device)
            preds.append(m(xb).cpu().numpy())
    y_pred_scaled = np.vstack(preds).ravel()
    y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).ravel()
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    mae  = float(mean_absolute_error(y_true, y_pred))
    r2   = float(r2_score(y_true, y_pred))
    return rmse, mae, r2

rmse_tr, mae_tr, r2_tr = evaluate_original_units(model, Xtr, ytr, y_train.ravel())
rmse_va, mae_va, r2_va = evaluate_original_units(model, Xva, yva, y_val.ravel())
rmse_te, mae_te, r2_te = evaluate_original_units(model, Xte, yte, y_test.ravel())

print("\n=== Metrics (original units) ===")
print(f"Train: RMSE {rmse_tr:.2f} | MAE {mae_tr:.2f} | R^2 {r2_tr:.4f}")
print(f"Val  : RMSE {rmse_va:.2f} | MAE {mae_va:.2f} | R^2 {r2_va:.4f}")
print(f"Test : RMSE {rmse_te:.2f} | MAE {mae_te:.2f} | R^2 {r2_te:.4f}")


Device: cuda


C:\Users\colli\AppData\Local\Temp\ipykernel_56612\170025695.py:66: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_amp = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))


Best epoch: 34 | val RMSE (scaled): 0.3808 | time: 0.77 min

=== Metrics (original units) ===
Train: RMSE 45.88 | MAE 24.56 | R^2 0.9982
Val  : RMSE 417.30 | MAE 251.40 | R^2 0.8516
Test : RMSE 419.74 | MAE 254.55 | R^2 0.8522


# NN Descriptor Analysis

In [15]:
# nn_grid_search_descriptors.py
import os, json, time, numpy as np, pandas as pd, torch, torch.nn as nn
from pathlib import Path
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

# ========= CONFIG =========
SEED = 42
np.random.seed(SEED); torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Path to descriptor CSV (col0=U0, col1=SMILES, rest=descriptors)
DESCRIPTOR_CSV = "qm9_desc_U0.csv"   # <--- change if needed

# Output folder
OUT = Path("artifacts_nn_desc"); OUT.mkdir(exist_ok=True)
CSV_RESULTS = OUT / "nn_val_search_stream_desc.csv"

# Small grid (tight, effective)
GRID = {
    "hidden":      [[512,256], [1024,512,256]],
    "act":         ["relu", "gelu"],
    "dropout":     [0.0, 0.1],
    "weight_decay":[1e-5, 1e-4],
    "lr":          [1e-3, 5e-4],
    "batch_size":  [512, 1024],
    "epochs":      [100],
    "patience":    [10],
}
# ==========================

# ========= LOAD DATA =========
df = pd.read_csv(DESCRIPTOR_CSV)
y_all = df.iloc[:,0].to_numpy(np.float32)
X_all = df.iloc[:,2:].to_numpy(np.float32)  # skip U0 and SMILES
print("Descriptor matrix shape:", X_all.shape)

# Same 80/10/10 split pattern (deterministic)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_all, y_all, test_size=0.2, random_state=SEED, shuffle=True
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=SEED, shuffle=True
)

# Scale X on train only; scale y for training stability
x_scaler = StandardScaler().fit(X_train)
Xtr = x_scaler.transform(X_train).astype(np.float32)
Xva = x_scaler.transform(X_val).astype(np.float32)
Xte = x_scaler.transform(X_test).astype(np.float32)

y_scaler = StandardScaler().fit(y_train.reshape(-1,1))
ytr = y_scaler.transform(y_train.reshape(-1,1)).astype(np.float32)
yva = y_scaler.transform(y_val.reshape(-1,1)).astype(np.float32)
yte = y_scaler.transform(y_test.reshape(-1,1)).astype(np.float32)

# ========= MODEL / TRAINING =========
class FFN(nn.Module):
    def __init__(self, d_in, hidden, act="relu", p=0.0):
        super().__init__()
        A = nn.ReLU if act=="relu" else nn.GELU
        layers=[]; prev=d_in
        for h in hidden:
            layers += [nn.Linear(prev,h), A(), nn.Dropout(p)]
            prev=h
        layers += [nn.Linear(prev,1)]
        self.net = nn.Sequential(*layers)
    def forward(self,x): return self.net(x)

def make_loaders(Xtr, ytr, Xva, yva, bs):
    ds_tr = TensorDataset(torch.from_numpy(Xtr), torch.from_numpy(ytr))
    ds_va = TensorDataset(torch.from_numpy(Xva), torch.from_numpy(yva))
    dl_tr = DataLoader(ds_tr, batch_size=bs, shuffle=True,  pin_memory=True)
    dl_va = DataLoader(ds_va, batch_size=max(bs,2048), shuffle=False, pin_memory=True)
    return dl_tr, dl_va

def train_one(cfg):
    hidden   = cfg["hidden"]
    act      = cfg["act"]
    p        = cfg["dropout"]
    wd       = cfg["weight_decay"]
    lr       = cfg["lr"]
    bs       = cfg["batch_size"]
    max_ep   = cfg["epochs"]
    patience = cfg["patience"]

    dl_tr, dl_va = make_loaders(Xtr, ytr, Xva, yva, bs)
    model = FFN(Xtr.shape[1], hidden, act, p).to(device)
    opt   = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=3)
    crit  = nn.MSELoss()
    scaler = torch.amp.GradScaler("cuda", enabled=(device.type=="cuda"))

    best = {"rmse_val": float("inf"), "state": None, "epoch": -1}
    wait = 0
    t0 = time.perf_counter()

    for ep in range(1, max_ep+1):
        model.train(); tr_loss=0.0; nb=0
        for xb,yb in dl_tr:
            xb,yb = xb.to(device), yb.to(device)
            opt.zero_grad(set_to_none=True)
            with torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
                pred = model(xb); loss = crit(pred,yb)
            scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
            tr_loss += loss.item(); nb += 1

        model.eval(); mse=0.0; n=0
        with torch.no_grad(), torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
            for xb,yb in dl_va:
                xb,yb = xb.to(device), yb.to(device)
                pred = model(xb)
                mse += ((pred-yb)**2).mean().item(); n += 1
        mse /= max(1,n); rmse_val = float(np.sqrt(mse))
        sched.step(mse)

        if rmse_val < best["rmse_val"] - 1e-6:
            best.update({"rmse_val": rmse_val, "state": {k:v.detach().cpu() for k,v in model.state_dict().items()}, "epoch": ep})
            wait = 0
        else:
            wait += 1
            if wait >= patience: break

    secs = time.perf_counter() - t0
    return best["rmse_val"], best["state"], secs, best["epoch"]

def cfg_iter(grid):
    keys = list(grid.keys())
    for vals in product(*[grid[k] for k in keys]):
        yield dict(zip(keys, vals))

# ========= STREAMING GRID SEARCH =========
done = set()
if CSV_RESULTS.exists():
    prev = pd.read_csv(CSV_RESULTS)
    for _,r in prev.iterrows():
        d = {k: r[k] for k in ["hidden","act","dropout","weight_decay","lr","batch_size","epochs","patience"]}
        done.add(json.dumps(d, sort_keys=True))

for cfg in cfg_iter(GRID):
    key = json.dumps(cfg, sort_keys=True)
    if key in done:
        print("Skip:", cfg); continue
    print("Train:", cfg)
    rmse_val, state, secs, best_ep = train_one(cfg)
    row = dict(cfg); row.update({"rmse_val": rmse_val, "secs": secs, "best_epoch": best_ep})
    pd.DataFrame([row]).to_csv(CSV_RESULTS, mode="a", header=not CSV_RESULTS.exists(), index=False)
    print(f"-> val RMSE (scaled) {rmse_val:.4f} | {secs/60:.2f} min | best ep {best_ep}")

# ========= PICK BEST, RETRAIN ON TRAIN+VAL, TEST =========
all_df = pd.read_csv(CSV_RESULTS).sort_values("rmse_val").reset_index(drop=True)
best = all_df.iloc[0].to_dict()
print("\nBest (descriptors):", best)

# Retrain on train+val combined
Xtv = np.vstack([Xtr, Xva]); ytv = np.vstack([ytr, yva])
bs = int(best["batch_size"])
dl_tv = DataLoader(TensorDataset(torch.from_numpy(Xtv), torch.from_numpy(ytv)),
                   batch_size=bs, shuffle=True, pin_memory=True)
dl_te = DataLoader(TensorDataset(torch.from_numpy(Xte), torch.from_numpy(yte)),
                   batch_size=max(bs,2048), shuffle=False, pin_memory=True)

model = FFN(
    Xtr.shape[1],
    hidden=eval(best["hidden"]) if isinstance(best["hidden"], str) else best["hidden"],
    act=best["act"],
    p=float(best["dropout"])
).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=float(best["lr"]), weight_decay=float(best["weight_decay"]))
crit = nn.MSELoss()
scaler = torch.amp.GradScaler("cuda", enabled=(device.type=="cuda"))

E = int(best["best_epoch"]) + 5
for ep in range(E):
    model.train()
    for xb,yb in dl_tv:
        xb,yb = xb.to(device), yb.to(device)
        opt.zero_grad(set_to_none=True)
        with torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
            pred = model(xb); loss = crit(pred,yb)
        scaler.scale(loss).backward(); scaler.step(opt); scaler.update()

# Test in original units
model.eval(); preds=[]
with torch.no_grad(), torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
    for xb,yb in dl_te:
        xb = xb.to(device)
        preds.append(model(xb).cpu().numpy())
y_pred_scaled = np.vstack(preds).ravel()
y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).ravel()
y_true = y_scaler.inverse_transform(yte).ravel()

def metrics(y_true, y_pred):
    rmse = float(np.sqrt(np.mean((y_pred-y_true)**2)))
    mae  = float(np.mean(np.abs(y_pred-y_true)))
    ss_res = np.sum((y_pred-y_true)**2); ss_tot = np.sum((y_true-y_true.mean())**2)
    r2 = float(1 - ss_res/ss_tot)
    return rmse, mae, r2

rmse_d, mae_d, r2_d = metrics(y_true, y_pred)
print(f"\n=== Descriptors NN Test ===\nRMSE {rmse_d:.6f} | MAE {mae_d:.6f} | R^2 {r2_d:.6f}")

# Save artifacts
import joblib
joblib.dump({"x_scaler": x_scaler, "y_scaler": y_scaler, "state_dict": model.state_dict(), "config": best},
            OUT / "nn_desc_best_model.joblib")
pd.DataFrame({"y_true": y_true, "y_pred": y_pred}).to_csv(OUT / "test_preds_desc.csv", index=False)

# ========= OPTIONAL: COMPARE TO FINGERPRINT NN IF AVAILABLE =========
fp_preds_path = Path("artifacts_nn/test_preds.csv")
if fp_preds_path.exists():
    df_fp = pd.read_csv(fp_preds_path)
    y_true_fp = df_fp["y_true"].to_numpy()
    y_pred_fp = df_fp["y_pred"].to_numpy()
    rmse_f, mae_f, r2_f = metrics(y_true_fp, y_pred_fp)

    comp = pd.DataFrame([
        {"model":"NN (fingerprints)", "RMSE":rmse_f, "MAE":mae_f, "R2":r2_f},
        {"model":"NN (descriptors) ", "RMSE":rmse_d, "MAE":mae_d, "R2":r2_d},
    ]).sort_values("RMSE")
    print("\n=== Comparison (lower is better) ===")
    print(comp.to_string(index=False))
    comp.to_csv(OUT / "nn_fp_vs_desc_metrics.csv", index=False)
else:
    print("\nNo fingerprint predictions found at artifacts_nn/test_preds.csv; "
          "skipping side-by-side comparison.")


Device: cuda
Descriptor matrix shape: (129012, 194)
Train: {'hidden': [512, 256], 'act': 'relu', 'dropout': 0.0, 'weight_decay': 1e-05, 'lr': 0.001, 'batch_size': 512, 'epochs': 100, 'patience': 10}
-> val RMSE (scaled) 0.0055 | 1.56 min | best ep 98
Train: {'hidden': [512, 256], 'act': 'relu', 'dropout': 0.0, 'weight_decay': 1e-05, 'lr': 0.001, 'batch_size': 1024, 'epochs': 100, 'patience': 10}
-> val RMSE (scaled) 0.0073 | 1.25 min | best ep 100
Train: {'hidden': [512, 256], 'act': 'relu', 'dropout': 0.0, 'weight_decay': 1e-05, 'lr': 0.0005, 'batch_size': 512, 'epochs': 100, 'patience': 10}
-> val RMSE (scaled) 0.0071 | 1.63 min | best ep 99
Train: {'hidden': [512, 256], 'act': 'relu', 'dropout': 0.0, 'weight_decay': 1e-05, 'lr': 0.0005, 'batch_size': 1024, 'epochs': 100, 'patience': 10}
-> val RMSE (scaled) 0.0092 | 1.35 min | best ep 97
Train: {'hidden': [512, 256], 'act': 'relu', 'dropout': 0.0, 'weight_decay': 0.0001, 'lr': 0.001, 'batch_size': 512, 'epochs': 100, 'patience': 10}

In [27]:
# nn_best_desc_retrain_matchgrid.py
import time, numpy as np, pandas as pd, torch, torch.nn as nn
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.utils.data import TensorDataset, DataLoader

# ========= CONFIG =========
SEED = 42
np.random.seed(SEED); torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

CSV = "qm9_desc_U0.csv"          # col0=U0, col1=SMILES, rest=descriptors
OUT = Path("artifacts_nn_desc"); OUT.mkdir(exist_ok=True)
PRED_CSV = OUT / "test_preds_desc.csv"
MODEL_PTH = OUT / "nn_desc_best_matchgrid.pth"

# Best from your search
hidden       = [1024, 512, 256]
act_name     = "gelu"            # {"relu","gelu"}
dropout_p    = 0.0
weight_decay = 1e-4
lr           = 1e-3
batch_size   = 512
epochs       = 100
patience     = 10
# ==========================

# ========= LOAD / SPLIT / SCALE (identical to grid) =========
df = pd.read_csv(CSV)
y_all = df.iloc[:,0].to_numpy(np.float32)
X_all = df.iloc[:,2:].to_numpy(np.float32)
print("Descriptor matrix shape:", X_all.shape)

X_train, X_temp, y_train, y_temp = train_test_split(
    X_all, y_all, test_size=0.2, random_state=SEED, shuffle=True
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=SEED, shuffle=True
)

x_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train.reshape(-1,1))

Xtr = x_scaler.transform(X_train).astype(np.float32)
Xva = x_scaler.transform(X_val).astype(np.float32)
Xte = x_scaler.transform(X_test).astype(np.float32)

# NOTE: keep y as (N,1) like the grid code
ytr = y_scaler.transform(y_train.reshape(-1,1)).astype(np.float32)
yva = y_scaler.transform(y_val.reshape(-1,1)).astype(np.float32)
yte = y_scaler.transform(y_test.reshape(-1,1)).astype(np.float32)

def make_loaders(Xtr, ytr, Xva, yva, bs):
    ds_tr = TensorDataset(torch.from_numpy(Xtr), torch.from_numpy(ytr))
    ds_va = TensorDataset(torch.from_numpy(Xva), torch.from_numpy(yva))
    dl_tr = DataLoader(ds_tr, batch_size=bs, shuffle=True,  pin_memory=True)
    dl_va = DataLoader(ds_va, batch_size=max(bs,2048), shuffle=False, pin_memory=True)
    return dl_tr, dl_va

train_loader, val_loader = make_loaders(Xtr, ytr, Xva, yva, batch_size)
test_loader = DataLoader(TensorDataset(torch.from_numpy(Xte), torch.from_numpy(yte)),
                         batch_size=max(batch_size,2048), shuffle=False, pin_memory=True)

# ========= MODEL (mirror grid) =========
class FFN(nn.Module):
    def __init__(self, d_in, hidden, act="relu", p=0.0):
        super().__init__()
        A = nn.ReLU if act=="relu" else nn.GELU
        layers=[]; prev=d_in
        for h in hidden:
            layers += [nn.Linear(prev,h), A(), nn.Dropout(p)]  # always include Dropout(p)
            prev=h
        layers += [nn.Linear(prev,1)]
        self.net = nn.Sequential(*layers)
    def forward(self,x): return self.net(x)

model = FFN(Xtr.shape[1], hidden, act=act_name, p=dropout_p).to(device)

opt   = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=3)
crit  = nn.MSELoss()
scaler = torch.amp.GradScaler("cuda", enabled=(device.type=="cuda"))

# ========= TRAIN (early stopping, same RMSE calc) =========
best_rmse = float("inf")
best_state = None
wait = 0
t0 = time.perf_counter()

for ep in range(1, epochs+1):
    model.train()
    for xb,yb in train_loader:
        xb,yb = xb.to(device), yb.to(device)
        opt.zero_grad(set_to_none=True)
        with torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
            pred = model(xb); loss = crit(pred,yb)
        scaler.scale(loss).backward(); scaler.step(opt); scaler.update()

    # val RMSE computed as mean of per-batch MSEs (grid style)
    model.eval(); mse=0.0; n=0
    with torch.no_grad(), torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
        for xb,yb in val_loader:
            xb,yb = xb.to(device), yb.to(device)
            pred = model(xb)
            mse += ((pred-yb)**2).mean().item(); n += 1
    mse /= max(1,n); rmse_val = float(np.sqrt(mse))
    sched.step(mse)

    print(f"Epoch {ep:03d} | val RMSE (scaled): {rmse_val:.6f}")

    if rmse_val < best_rmse - 1e-6:
        best_rmse = rmse_val
        best_state = {k:v.detach().cpu() for k,v in model.state_dict().items()}
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print(f"Early stopping at epoch {ep}."); break

print(f"Training time: {time.perf_counter()-t0:.1f}s")
model.load_state_dict(best_state)
torch.save(model.state_dict(), MODEL_PTH)

# ========= TEST (invert scaling; metrics on original units) =========
def eval_to_units(loader):
    model.eval(); P=[]; T=[]
    with torch.no_grad(), torch.amp.autocast("cuda", enabled=(device.type=="cuda")):
        for xb,yb in loader:
            xb = xb.to(device)
            P.append(model(xb).cpu().numpy())
            T.append(yb.numpy())
    P = np.vstack(P).ravel(); T = np.vstack(T).ravel()
    P_u = y_scaler.inverse_transform(P.reshape(-1,1)).ravel()
    T_u = y_scaler.inverse_transform(T.reshape(-1,1)).ravel()
    rmse = np.sqrt(mean_squared_error(T_u, P_u))
    mae  = mean_absolute_error(T_u, P_u)
    r2   = r2_score(T_u, P_u)
    return rmse, mae, r2, T_u, P_u

# Train/Val/Test metrics in original units
rmse_tr, mae_tr, r2_tr, _, _ = eval_to_units(train_loader)
rmse_va, mae_va, r2_va, _, _ = eval_to_units(val_loader)
rmse_te, mae_te, r2_te, y_true, y_pred = eval_to_units(test_loader)

print("\n=== Final Metrics (Descriptor NN; match-grid) ===")
print(f"Train: RMSE={rmse_tr:.6f} | MAE={mae_tr:.6f} | R2={r2_tr:.6f}")
print(f"Val:   RMSE={rmse_va:.6f} | MAE={mae_va:.6f} | R2={r2_va:.6f}")
print(f"Test:  RMSE={rmse_te:.6f} | MAE={mae_te:.6f} | R2={r2_te:.6f}")

pd.DataFrame({"y_true": y_true, "y_pred": y_pred}).to_csv(PRED_CSV, index=False)
print(f"Saved test predictions → {PRED_CSV}")
print(f"Saved model → {MODEL_PTH}")


Device: cuda
Descriptor matrix shape: (129012, 194)
Epoch 001 | val RMSE (scaled): 0.037732
Epoch 002 | val RMSE (scaled): 0.040970
Epoch 003 | val RMSE (scaled): 0.023502
Epoch 004 | val RMSE (scaled): 0.023857
Epoch 005 | val RMSE (scaled): 0.038393
Epoch 006 | val RMSE (scaled): 0.022954
Epoch 007 | val RMSE (scaled): 0.030329
Epoch 008 | val RMSE (scaled): 0.188664
Epoch 009 | val RMSE (scaled): 0.057578
Epoch 010 | val RMSE (scaled): 0.026409
Epoch 011 | val RMSE (scaled): 0.018251
Epoch 012 | val RMSE (scaled): 0.016680
Epoch 013 | val RMSE (scaled): 0.014384
Epoch 014 | val RMSE (scaled): 0.013559
Epoch 015 | val RMSE (scaled): 0.019765
Epoch 016 | val RMSE (scaled): 0.016011
Epoch 017 | val RMSE (scaled): 0.014517
Epoch 018 | val RMSE (scaled): 0.012629
Epoch 019 | val RMSE (scaled): 0.011586
Epoch 020 | val RMSE (scaled): 0.015068
Epoch 021 | val RMSE (scaled): 0.011334
Epoch 022 | val RMSE (scaled): 0.011403
Epoch 023 | val RMSE (scaled): 0.011770
Epoch 024 | val RMSE (scaled